In [1]:
from tevatron.datasets.dataset import load_dataset

2022-11-07 01:29:52.935977: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
dataset = load_dataset("Tevatron/msmarco-passage",
                      "default",
                      data_files=None, cache_dir=None)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset ms_marco_passage downloaded and prepared to /home/y247xie/.cache/huggingface/datasets/Tevatron___ms_marco_passage/default/0.0.1/1874f5d9ae5257b9dbc7d8f89c76f8d4c321be6b660bb5df208e5e64decfa978. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')


/home/y247xie/miniconda3/envs/pygaggle/lib/python3.8/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [27]:
import tqdm
qid2query_embedding = {}
query_batch = []
qid_batch = []
queries_msmarco = []
qids_msmarco = []
embeddings_msmarco = []
for data in tqdm.tqdm(dataset['train']):
    query_batch.append(data['query'])
    qid_batch.append(data['query_id'])
    queries_msmarco.append(data['query'])
    qids_msmarco.append(data['query_id'])
    if len(qid_batch) == 512:
        query_embeddings = model.encode(query_batch)
        for i, qid in enumerate(qid_batch):
            qid2query_embedding[qid] = query_embeddings[i]
            embeddings_msmarco.append(query_embeddings[i])
        query_batch = []
        qid_batch = []
            
if len(qid_batch) > 0 :
    query_embeddings = model.encode(query_batch)
    for i, qid in enumerate(qid_batch):
        qid2query_embedding[qid] = query_embeddings[i]
        embeddings_msmarco.append(query_embeddings[i])
        


100%|█████████████████████████████| 400782/400782 [18:23<00:00, 363.31it/s]


In [13]:
dataset_scifact = load_dataset("Tevatron/scifact",
                      "default",
                      data_files=None, cache_dir=None)

Reusing dataset scifact (/home/y247xie/.cache/huggingface/datasets/Tevatron___scifact/default/0.0.1/0407c8eb5881d4870a4474489aeb475222ebc3febda54c236dc871ea69cd85b2)


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
qid2query_embedding_scifact = {}
for data in tqdm.tqdm(dataset_scifact['dev']):
    query_embeddings = model.encode(data['query'])
    qid2query_embedding_scifact[data['query_id']] = query_embeddings

In [19]:
import numpy as np
embedding_scifact = sum(qid2query_embedding_scifact[qid] for qid in qid2query_embedding_scifact) / len(qid2query_embedding_scifact)

In [131]:
embeddings_scifact = np.array([qid2query_embedding_scifact[qid] for qid in qid2query_embedding_scifact])

In [138]:
total = 0
count = 0
scifact_sims = []
for i, sims in enumerate(np.dot(embeddings_scifact, embeddings_scifact.T)):
    total += sum(sims) - sims[i]
    count += len(sims) - 1
    scifact_sims += list(sims[:i]) + list(sims[i+1:])
    


In [140]:
pd.DataFrame(scifact_sims).describe()

,0
count,89700.000000
mean,0.123611
std,0.113261
min,-0.176934
25%,0.040030
50%,0.110267
75%,0.194651
max,0.998386


In [22]:
import numpy as np
scores = np.dot(embedding_scifact, np.array(embeddings_msmarco).T)

In [34]:
sim_pids = [(queries_msmarco[i], score, qids_msmarco[i]) for i, score in enumerate(scores)]
sim_pids = sorted(sim_pids, key=lambda x: x[1], reverse=True)

In [35]:
sim_pids[:100]

[('_____ can selectively enhance the growth and proliferation of mutated cells.',
  0.20802751,
  '8653'),
 ('Which is a part of the innate immune defenses', 0.19895679, '8219'),
 ('what cells does tbev effect', 0.19841647, '594203'),
 ('what organelle is most active in causing programmed cell death',
  0.19757584,
  '883402'),
 ('what is the function in humans of the protein produced from that gene',
  0.19436993,
  '822257'),
 ('is a disease-fighting protein created', 0.19391418, '399823'),
 ('role of nf kb in inflammatory disease', 0.19186991, '489746'),
 ('what processes are g0 cells unable to do', 0.19105671, '888744'),
 ('what is the ret gene associated with?', 0.1893949, '844271'),
 ('why is receptor mediated endocytosis important', 0.18610172, '1071929'),
 ('inflammation is accompanied by', 0.18576144, '395826'),
 ('what attacks helper t cells?', 0.18470383, '1165309'),
 ('what do nucleosomes inhibit?', 0.18462797, '1162925'),
 ('RNA and polypeptides interact with what cell loc

In [43]:
sorted_qids = [qid for _, _, qid in sim_pids]
qid2score = {qid: score for _, score, qid in sim_pids}

In [96]:
import pickle
pickle.dump(qid2score, open("qid2score.pkl", "wb"))

In [51]:
for i in range(len(dataset['train'])):
    dataset['train'][i]['score'] = qid2score[dataset['train'][i]['query_id']]

In [99]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [102]:
dataset['train'][1]

{'query_id': '1000684',
 'query': 'where is your perineum',
 'positive_passages': [{'docid': '6133670',
   'title': 'perineum',
   'text': 'That part of the floor of the PELVIS that lies between the tops of the thighs. In the male, the perineum lies between the anus and the scrotum. In the female, it includes the external genitalia. The area between the opening of the vagina and the anus in a woman, or the area between the scrotum and the anus in a man.'}],
 'negative_passages': [{'docid': '6475882',
   'title': 'After the birth',
   'text': 'To help the perineum to heal. 1  Keep the area cold by using a wrapped ice pack for the first 24 to 72 hours. 2  This decreases the pain and swelling which is important for healing and comfort. 3  Have the wrapped ice pack on for 10 to 20 minutes at the most each time. 4  Change sanitary pads often to keep the perineum dry.'},
  {'docid': '8032962',
   'title': 'Basal-cell carcinoma',
   'text': 'Aberrant basal cell carcinoma is characterized by t

In [68]:
def add_score(example):
    example["score"] = qid2score[example['query_id']]
    return example

dataset_train = dataset['train'].map(add_score)

  0%|          | 0/400782 [00:00<?, ?ex/s]

In [104]:
dataset['train'][1]

{'query_id': '1000684',
 'query': 'where is your perineum',
 'positive_passages': [{'docid': '6133670',
   'title': 'perineum',
   'text': 'That part of the floor of the PELVIS that lies between the tops of the thighs. In the male, the perineum lies between the anus and the scrotum. In the female, it includes the external genitalia. The area between the opening of the vagina and the anus in a woman, or the area between the scrotum and the anus in a man.'}],
 'negative_passages': [{'docid': '6475882',
   'title': 'After the birth',
   'text': 'To help the perineum to heal. 1  Keep the area cold by using a wrapped ice pack for the first 24 to 72 hours. 2  This decreases the pain and swelling which is important for healing and comfort. 3  Have the wrapped ice pack on for 10 to 20 minutes at the most each time. 4  Change sanitary pads often to keep the perineum dry.'},
  {'docid': '8032962',
   'title': 'Basal-cell carcinoma',
   'text': 'Aberrant basal cell carcinoma is characterized by t

In [74]:
dataset_train = dataset_train.sort('score')

Loading cached sorted indices for dataset at /home/y247xie/.cache/huggingface/datasets/Tevatron___ms_marco_passage/default/0.0.1/1874f5d9ae5257b9dbc7d8f89c76f8d4c321be6b660bb5df208e5e64decfa978/cache-852e6bcd236126db.arrow


In [118]:
int(len(dataset_train)/10)

40078

In [81]:
dataset_train_1  = dataset_train[:int(len(dataset_train)/10)]

In [114]:
i = 0
data_test = dataset_train._getitem(slice(i, i + segment_size), decoded=False)

In [116]:
data_test.keys()

dict_keys(['query_id', 'query', 'positive_passages', 'negative_passages', 'score'])

In [125]:
dataset_train[1]['score']

-0.07216934859752655

In [145]:
dataset_segments_train = []

score_segments = [0.03, 0.1, 1]
score_segments_i = 0
# segment_size = int(len(dataset_train)/10) + 1
pre_i = 0
for i in range(len(dataset_train)):
    if dataset_train[i]['score'] > score_segments[score_segments_i]:
        dataset_segments_train.append(dataset_train[pre_i:i])
        score_segments_i += 1
        pre_i = i
        
dataset_segments_train.append(dataset_train[pre_i:])

In [146]:
[len(d['query']) for d in dataset_segments_train]

[319462, 75136, 6184]

In [89]:
# evenly split dataset, doesn't work with learning rate warm up and decay

# dataset_segments_train = []

# segment_size = int(len(dataset_train)/10) + 1
# for i in range(0, len(dataset_train), segment_size):
#     dataset_segments_train.append(dataset_train[i:i+segment_size])

In [119]:
len(dataset_segments_train)

11

In [108]:
dataset_segments_train[2].keys()

dict_keys(['query_id', 'query', 'positive_passages', 'negative_passages', 'score'])

In [83]:
len(dataset_train_1), int(len(dataset_train)/10)

(5, 40078)

In [87]:
len(dataset_train_1['query_id'])

40078

In [80]:
import pandas as pd
pd.DataFrame([score for _, score, qid in sim_pids]).describe()

,0
count,400782.000000
mean,0.012950
std,0.028777
min,-0.075672
25%,-0.005908
50%,0.006377
75%,0.023939
max,0.208028


In [123]:
sum([score > 0 and score < 0.1 for _, score, qid in sim_pids]) 

246996